In [2]:
from sklearn.datasets import load_iris #dataset para la validación 
from sklearn.model_selection import train_test_split #Separacón de variables xtrain 
X, y = load_iris(return_X_y=True)# cargamos el dataset
x_train, x_val, y_train, y_val = train_test_split(X, y, random_state=0)# genermos la separación 

# determinamos el path donde se alojara la base de oracle y json de keras tune como un directorio
import os 
path=os.chdir('c:\\Users\\constantinoj\\Desktop\\Prueba_keras_tune')# este directorio tambien puede cambiar 

import numpy as np #Calculo cientifico eficiente 
from tensorflow import keras# framework de creación 
from tensorflow.keras import layers# obtenemos las capas 
from keras_tuner.tuners import RandomSearch# obtenemos el metodo de busqueda ( falta comparar como busqueda bayeciana )
from sklearn.model_selection import GridSearchCV# Busqueda de hiperparametros por maya epocas ( con el modelo ya dimencionado)
from sklearn.preprocessing import StandardScaler  #Normalizamos la data ( solo para este ejemplo ( NLP es distinto))
from scikeras.wrappers import KerasClassifier


In [3]:
# Validamos los valores unicos para evitar que existan valores distintos en el proceso de spliteo
print(np.unique(y_train))
print(np.unique(y_val))

[0 1 2]
[0 1 2]


In [4]:
# Se presenta un ejemplo de arquitectura propuesta solo como documentación de como se redimencionan las capas de la ANN 
def build_modelo(hp):
    modelo = keras.Sequential()
        # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 9)):
        modelo.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
    
    if hp.Boolean("dropout"):
        modelo.add(layers.Dropout(rate=0.25))
    
    modelo.add(layers.Dense(3, activation="softmax"))

    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    modelo.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return modelo

tuner = RandomSearch(
    build_modelo,
    objective='val_accuracy',# 
    max_trials=7,
    executions_per_trial=4,
    overwrite=True,
    directory=path,
    project_name="Primer_acercamiento",)

tuner.search(x_train, y_train,
             epochs=5,# numero de modelos 
             validation_data=(x_val, y_val))

Trial 7 Complete [00h 00m 14s]
val_accuracy: 0.9078947454690933

Best val_accuracy So Far: 0.9473684281110764
Total elapsed time: 00h 01m 36s


In [5]:
# validamos en donde se encuentra 
tuner.directory

'.'

In [6]:
# validamos el formato del path
type(path)

NoneType

In [7]:
# Ejercicio de muestra como busca en las diferentes arquitecturas hasta quedarse con la mas eficiente 
for i in range(len(tuner.get_best_models(num_models=4))):
    tuner.get_best_models(num_models=4)[i].summary()

c:\Users\constantinoj\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
c:\Users\constantinoj\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
c:\Users\constantinoj\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
c:\Users\constantinoj\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 160)            │        41,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 416)            │        66,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 384)            │       160,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 384)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │         1,155 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 303,043 (1.16 MB)

 Trainable params: 303,043 (1.16 MB)

...

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 352)            │         1,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 192)            │        67,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           579 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 70,115 (273.89 KB)

 Trainable params: 70,115 (273.89 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Nos quedamos con el mejor modelo
best_model1 = tuner.get_best_models(num_models)[0]

NameError: name 'num_models' is not defined

In [ ]:
# observamos el mejor comportamiento del modelo
best_model1.summary()

NameError: name 'best_model' is not defined

In [ ]:
# De las descripciones largas se obtiene el histograma para la desición del tamaño inical del embeding 
NUM_PALABRAS=2000


# construimos el modelo
def build_model_clasif_gen(hp4):
    # El parametro (hp4 )que va a crear un diccionario de los numeros de 
    # Iniciamos la red neuronal sin capas 
    modelo=keras.Sequential()
    # Agregamos la capa de enbedding
    modelo.add(layers.Embedding(NUM_PALABRAS,output_dim=hp4.Int("units",min_value=60, max_value=300,step=60)))
    # Desactivamos un porcentaje de las 
    modelo.add(layers.Dropout(0.1))
    # Aplanamos la estructura en un vector largo 
    modelo.add(layers.Flatten())
    # Agregamos una capa totalmente conectada 
    modelo.add(layers.Dense(30,activation="relu"))
    # aplicamos capa de desactivacion (porcentaje )
    modelo.add(layers.Dropout(0.1))
    # Agregamos una capa totalmente conectada 
    modelo.add(layers.Dense(12, activation="relu"))
    # Agregamos capa de olvido
    modelo.add(layers.Dropout(0.25))
    # Agregamos capa de salida
    modelo.add(layers.Dense(3, activation='softmax'))
    
    # Programamos los hiperparametros 
    learning_rate = hp4.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")

    # compliacion del modelo
    modelo.compile(
        # Importamos el Optimizador de adam ( con el buscador de hiperparametros)
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="sparse_categorical_crossentropy", # Perdida asociada a la forma de los 
        metrics=["accuracy"]) # Precicion de la clasificacion 
    return modelo

# creamos el tipo de busqueda aleatoria
tuner1 = RandomSearch(
    build_model_clasif_gen,# pasamos la arquitectura
    objective='val_accuracy',# me damos el metodo al buscador de Hiperparametros 
    max_trials=5,# maximo de intentos
    executions_per_trial=6,# numero de intentos por iteración 
    overwrite=False, # Sobre escritura 
    directory=path, # Direccion donde se almacenaran los resultados
    project_name=path)# Nombre del proyecto 

In [ ]:
tuner1.search(x_train, #Variables predictoras
              y_train,# Variables predictoras 
             epochs=2,# numero de epocas ( Este parametro se optimizara mas adelante )
             validation_data=(x_val, y_val))# evitamos el overfiting dentro de los datos 

Trial 5 Complete [00h 00m 18s]
val_accuracy: 0.8815789421399435

Best val_accuracy So Far: 0.8815789421399435
Total elapsed time: 00h 01m 27s


In [ ]:
# validamos el lugar donde esta el modelo
tuner.directory

'.'

In [ ]:
# Nos quedamos con el modelo que mejoro con mejor precicion 
best_model = tuner1.get_best_models(num_models=1)[0]

c:\Users\constantinoj\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# validamos la arquitectura encontrada por keras-tuner ( la que tiene mejores parametros )
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 4, 240)         │        24,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4, 240)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 960)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 30)             │        28,830 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 12)             │           372 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 12)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            39 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,241 (207.97 KB)

 Trainable params: 53,241 (207.97 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# guardamos el modelo en formato h5 ( en este caso se guarda en la raiz donde se encuente el archivo)
best_model.save("ia_clasificadora_generos.h5")